In [1]:
'''
Notebook to create confidence-interval plots for runs (Fig 4)
For debugging purposes, also generates overlay plot for individual runs
'''

import os

import cmws
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from cmws import util
from cmws.examples.scene_understanding import data, render, run, plot
from cmws.examples.scene_understanding import util as scene3d_util 
import seaborn as sns
import numpy as np
from cmws.examples.scene_understanding.plot import plot_with_error_bars, get_analysis_plots

In [6]:
# run cell to generate plots in Fig 4

label_map = {"cmws_5": "HMWS", "rws": "RWS",
            "reinforce": "REINFORCE", "vimco_2": "VIMCO"}

# get figures for paper (Fig 4)
def get_formal_overlay_plots(experiment_name, file_tag, grid_sizes=[2, 3], algorithms=["cmws_2", "cmws_4", "rws"],
                      aggregate_shrink=True, shrink_factors=[0.01], num_iters_show=5000):
    save_dir = f"../save/{experiment_name}"
    checkpoint_paths = []
    for config_name in sorted(os.listdir(save_dir)):
        checkpoint_paths.append(util.get_checkpoint_path(experiment_name, config_name, -1))

    for grid_size in grid_sizes:  # grid size
        for shrink_factor in shrink_factors: 
            # Load
            x = []
            log_ps = {alg: [] for idx, alg in enumerate(algorithms)}
            kls = {alg: [] for idx, alg in enumerate(algorithms)}
            colors = {"cmws_5": "C0", "rws": "C1", "vimco": "C2", "reinforce": "C3", "vimco_2": "C4"}
            #colors = {alg: f'C{idx}' for idx, alg in enumerate(algorithms)}
            for checkpoint_path in checkpoint_paths:
                checkpoint_path = f"../{checkpoint_path}"
                if os.path.exists(checkpoint_path):
                    # Load checkpoint
                    try:
                        model, optimizer, stats, run_args = scene3d_util.load_checkpoint(
                            checkpoint_path, device="cpu"
                        )
                    except:
                        continue
                    if run_args.num_grid_cols != grid_size: continue
                    if run_args.shrink_factor != 0.01: continue # PATCH
                    if not aggregate_shrink and run_args.shrink_factor != shrink_factor: continue 
                    if run_args.algorithm not in set(algorithms): continue 
                    #if len(stats.losses) < num_iters_show: continue

                    x_new = [x[0] for x in stats.log_ps]
                    if len(x_new) > len(x):
                        x = x_new
                    log_ps[run_args.algorithm].append([x[1] for x in stats.log_ps])
                    kls[run_args.algorithm].append([x[1] for x in stats.kls])
            # Make numpy arrays
            max_len = len(x)
            if grid_size == 2:
                num_seeds = 5  # 10
            else:
                num_seeds = 5
            log_ps_np = dict(
                [[algorithm, np.full((num_seeds, max_len), np.nan)] for algorithm in algorithms]
            )
            kls_np = dict([[algorithm, np.full((num_seeds, max_len), np.nan)] for algorithm in algorithms])
            for algorithm in algorithms:
                for seed in range(num_seeds):
                    try:
                        log_p = log_ps[algorithm][seed]
                        kl = kls[algorithm][seed]
                    except Exception:
                        log_p = []
                        kl = []
                    log_ps_np[algorithm][seed][: len(log_p)] = log_p
                    kls_np[algorithm][seed][: len(kl)] = kl

            # Plot
            fig, ax = plt.subplots(1, 1, figsize=(6, 4))
            for algorithm in algorithms:
                label = label_map[algorithm]
                linestyle = "solid"
                color = colors[algorithm]
                plot_kwargs = {"color": color, "linestyle": linestyle, "label": label}

                # Logp
                log_p = log_ps_np[algorithm]
                plot_with_error_bars(ax, x, log_p, **plot_kwargs)

            ax.set_xlim([0,num_iters_show])
            ax.set_xticks([0, num_iters_show])
            ax.set_xlabel("Iteration", labelpad=-10)
            ax.set_ylabel(f"$\\log p_\\theta(x)$")
            ax.legend(loc='lower right')
            sns.despine(ax=ax, trim=True)

            util.save_fig(fig, f"{save_dir}/losses_{file_tag}_overlay.pdf", dpi=400)
            
experiment_names = ["cmws_vs_rws_learnColor","cmws_vs_rws_noColor"]
file_tags = ["learnColor", "noColor"]
num_iters_show=5000
for experiment_name,file_tag in zip(experiment_names,file_tags): 
    get_formal_overlay_plots(experiment_name, file_tag, grid_sizes=[2], algorithms=["cmws_5", "rws", "vimco_2","reinforce"],
                       num_iters_show=num_iters_show)

In [3]:
# for debugging individual runs

experiment_names = ["cmws_vs_rws_noColor", "cmws_vs_rws_learnColor"]

for experiment_name in experiment_names: 
    get_analysis_plots(experiment_name, grid_sizes=[2], algorithms=["cmws_5", "rws", "vimco_2", "reinforce"])

    

color status:  True
path:  /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/block_0.01/colorless/train.pt
11:29:52 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:348 | INFO: Loading dataset (test = False)...
11:29:52 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:352 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/block_0.01/colorless/train.pt
11:29:52 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory


100%|██████████| 100/100 [00:00<00:00, 967.97it/s]

color status:  True
path:  /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/block_0.01/colorless/train.pt
11:29:52 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:348 | INFO: Loading dataset (test = False)...
11:29:52 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:352 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/block_0.01/colorless/train.pt
11:29:52 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory



100%|██████████| 100/100 [00:00<00:00, 1013.30it/s]


color status:  True
path:  /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/block_0.01/colorless/train.pt
11:29:52 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:348 | INFO: Loading dataset (test = False)...
11:29:52 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:352 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/block_0.01/colorless/train.pt
11:29:52 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory


...

11:33:30 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:352 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/cube_0.01/train.pt
11:33:30 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory


100%|██████████| 100/100 [00:00<00:00, 301.23it/s]


11:33:32 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/util.py:127 | INFO: Error Ran out of input
11:33:32 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/util.py:129 | INFO: Waiting for 1 seconds
11:33:33 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/util.py:127 | INFO: Error Ran out of input
11:33:33 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/util.py:129 | INFO: Waiting for 2 seconds
11:33:35 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/util.py:127 | INFO: Error Ran out of input
11:33:35 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/util.py:129 | INFO: Waiting for 4 seconds
11:33:40 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/util.py:127 | INFO: Error Ran out of input
11:33:40 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/util.py:129 | INFO: Waiting for 1 seconds
11:33:41 | /om/user/katiemc/continuous_mws/c

/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolat

11:34:05 | /om/user/katiemc/continuous_mws/cmws/util.py:293 | INFO: Saved to ../save/cmws_vs_rws_learnColor/losses_2_overlay.png


In [3]:
# # get unaggregated shrink_factors
# experiment_name = "cmws_vs_rws_learnColor_diffShrink"
# shrink_factors = [0.02, 0.05, 0.1, 0.2]
# get_analysis_plots(experiment_name, grid_sizes=[2], algorithms=["cmws_4", "rws"],
#                   aggregate_shrink=False, shrink_factors=shrink_factors)
